In [1]:
from tensorflow.compat.v1 import ConfigProto
from tensorflow.compat.v1 import InteractiveSession
config = ConfigProto()
config.gpu_options.allow_growth = True
session = InteractiveSession(config=config)

https://www.youtube.com/watch?v=1-NYPQw5THU&feature=youtu.be

In [2]:
import pandas as pd
import numpy as np
import datetime
from pandas_summary import DataFrameSummary

In [3]:
df = pd.read_feather('train_normalized_data.fth')
df_test = pd.read_feather('test_normalized_data.fth')

In [4]:
#cat_vars = ['Store', 'DayOfWeek', 'Year', 'Month', 'Day', 'StateHoliday', 'CompetitionMonthsOpen', 'Promo2Weeks', 'StoreType', 'Assortment', 'PromoInterval', 'CompetitionOpenSinceYear', 'Promo2SinceYear', 'State', 'Week', 'Events', 'Promo_fw', 'Promo_bw', 'StateHoliday_fw', 'StateHoliday_bw', 'SchoolHoliday_fw', 'SchoolHoliday_bw']

# cat_vars = ['Store', 'DayOfWeek', 'Year', 'Month', 'Day', 'State']

cat_vars = ['Store', 'DayOfWeek', 'Year', 'Month', 'Day', 'StateHoliday', 'CompetitionMonthsOpen', 'Promo2Weeks', 
            'StoreType', 'Assortment', 'PromoInterval', 'CompetitionOpenSinceYear', 'Promo2SinceYear', 'State', 
            'Week', 'Events', 'Promo_fw', 'Promo_bw', 'StateHoliday_bool_fw', 'StateHoliday_bool_bw', 'SchoolHoliday_fw', 'SchoolHoliday_bw']

In [5]:
#df.columns.to_list()

In [6]:
#contin_vars = ['CompetitionDistance', 
#   'Max_TemperatureC', 'Mean_TemperatureC', 'Min_TemperatureC',
#   'Max_Humidity', 'Mean_Humidity', 'Min_Humidity', 'Max_Wind_SpeedKm_h', 
#   'Mean_Wind_SpeedKm_h', 'CloudCover', 'trend', 'trend_DE',
#   'AfterStateHoliday', 'BeforeStateHoliday', 'Promo', 'SchoolHoliday']
# contin_vars = []
contin_vars = ['CompetitionDistance', 
   'Max_TemperatureC', 'Mean_TemperatureC', 'Min_TemperatureC', 'Precipitationmm',
   'Max_Humidity', 'Mean_Humidity', 'Min_Humidity', 'Max_Wind_SpeedKm_h', 
   'Mean_Wind_SpeedKm_h', 'CloudCover', 'trend', 'trend_DE',
   'AfterStateHoliday_bool', 'BeforeStateHoliday_bool', 'Promo', 'SchoolHoliday', 'StateHoliday_bool']

In [7]:
from lightgbm import LGBMRegressor

In [8]:
y_out_columns = ['Sales']

In [9]:
df_train = df[df.Date < datetime.datetime(2015, 7, 1)]  
df_val = df[df.Date >= datetime.datetime(2015, 7, 1)]
print(f'Cantidad en val: {len(df_val)}, porcentaje: {len(df_train)/(len(df_train) + len(df_val))}')

Cantidad en val: 30029, porcentaje: 0.9638795367092596


In [10]:
X_train = df_train[cat_vars + contin_vars]
X_val = df_val[cat_vars + contin_vars]
X_test = df_test[cat_vars + contin_vars]

In [11]:
X_train.shape, X_val.shape

((801328, 40), (30029, 40))

In [12]:
log_output = True
    
if log_output:
    # Escala logaritmica
    max_log_y = np.max(np.log(df[y_out_columns])).values
    y_train = np.log(df_train[y_out_columns].values)/max_log_y
    y_val = np.log(df_val[y_out_columns].values)/max_log_y
else:
    # Normalización
    y_mean = df_train[y_out_columns].mean().values
    y_std = df_train[y_out_columns].std().values
    y_train = (df_train[y_out_columns].values - y_mean)/y_std
    y_val = (df_val[y_out_columns].values - y_mean)/y_std

In [13]:
min_child_samples=5
n_estimators=4000
learning_rate=0.05
model = LGBMRegressor(min_child_samples=min_child_samples, n_estimators=n_estimators, learning_rate=learning_rate )

In [14]:
fit_params={"early_stopping_rounds":100, 
            "eval_metric" : 'l2', 
            "eval_set" : [(X_val, y_val.reshape(-1))],
            'eval_names': ['valid'],
            'verbose': 100,
            'feature_name': 'auto', # that's actually the default
            'categorical_feature': cat_vars
           }

In [15]:
model.fit(X_train, y_train.reshape(-1), **fit_params)

C:\Users\fvillarino\Anaconda3\envs\dip-ml-env\lib\site-packages\lightgbm\basic.py:1551: UserWarning: Using categorical_feature in Dataset.
  warnings.warn('Using categorical_feature in Dataset.')
C:\Users\fvillarino\Anaconda3\envs\dip-ml-env\lib\site-packages\lightgbm\basic.py:1554: UserWarning: categorical_feature in Dataset is overridden.
New categorical_feature is ['Assortment', 'CompetitionMonthsOpen', 'CompetitionOpenSinceYear', 'Day', 'DayOfWeek', 'Events', 'Month', 'Promo2SinceYear', 'Promo2Weeks', 'PromoInterval', 'Promo_bw', 'Promo_fw', 'SchoolHoliday_bw', 'SchoolHoliday_fw', 'State', 'StateHoliday', 'StateHoliday_bool_bw', 'StateHoliday_bool_fw', 'Store', 'StoreType', 'Week', 'Year']
  warnings.warn('categorical_feature in Dataset is overridden.\n'
C:\Users\fvillarino\Anaconda3\envs\dip-ml-env\lib\site-packages\lightgbm\basic.py:1286: UserWarning: Overriding the parameters from Reference Dataset.
  warnings.warn('Overriding the parameters from Reference Dataset.')
C:\Users\fv

Training until validation scores don't improve for 100 rounds
[100]	valid's l2: 0.000230447
[200]	valid's l2: 0.000173992
[300]	valid's l2: 0.00015348
[400]	valid's l2: 0.000145573


ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.



Traceback (most recent call last):
  File "C:\Users\fvillarino\Anaconda3\envs\dip-ml-env\lib\site-packages\IPython\core\interactiveshell.py", line 3417, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "<ipython-input-15-d7f26861f2f8>", line 1, in <module>
    model.fit(X_train, y_train.reshape(-1), **fit_params)
  File "C:\Users\fvillarino\Anaconda3\envs\dip-ml-env\lib\site-packages\lightgbm\sklearn.py", line 770, in fit
    super(LGBMRegressor, self).fit(X, y, sample_weight=sample_weight,
  File "C:\Users\fvillarino\Anaconda3\envs\dip-ml-env\lib\site-packages\lightgbm\sklearn.py", line 612, in fit
    self._Booster = train(params, train_set,
  File "C:\Users\fvillarino\Anaconda3\envs\dip-ml-env\lib\site-packages\lightgbm\engine.py", line 252, in train
    booster.update(fobj=fobj)
  File "C:\Users\fvillarino\Anaconda3\envs\dip-ml-env\lib\site-packages\lightgbm\basic.py", line 2458, in update
    _safe_call(_LIB.LGBM_BoosterUpdateOneIter(
KeyboardInterrupt

Dur

TypeError: object of type 'NoneType' has no len()

# Métrica

$$
\textrm{RMSE} = \sqrt{\frac{1}{n} \sum_{i=1}^{n} \left(\frac{\hat{y}_i - y_i}{y_i}\right)^2}
$$

In [ ]:
model.score(X_val, y_val)

In [ ]:
if log_output:
    y_pred_train = np.exp(model.predict(X_train, verbose=1)*max_log_y)
    y_pred = np.exp(model.predict(X_val, verbose=1)*max_log_y)
    y_pred_test = np.exp(model.predict(X_test, verbose=1)*max_log_y)
else:
    y_pred_train = model.predict(X_train, verbose=1)*y_std + y_mean
    y_pred = model.predict(X_val, verbose=1)*y_std + y_mean
    y_pred_test = model.predict(X_test, verbose=1)*y_std + y_mean

In [ ]:
# Train
np.sqrt((((df_train['Sales'].values - y_pred_train)/df_train['Sales'].values)**2).sum()/len(y_pred_train))

In [ ]:
# Validación
np.sqrt((((df_val['Sales'].values - y_pred)/df_val['Sales'].values)**2).sum()/len(y_pred))

# Baseline

In [20]:
import pandas as pd
sample_csv = pd.read_csv('dataset/rossmann/sample_submission.csv')

In [21]:
stores_mean = {}
for store, g_df in df.groupby('Store'):
    stores_mean[store] = g_df[g_df['Sales'] > 0]['Sales'].mean()

In [22]:
df_test['Sales'] = df_test['Store'].apply(stores_mean.get)
df_test.loc[df_test['Open'] == 0, 'Sales'] = 0

In [23]:
df_test[['Store', 'Sales']].head(10)

,Store,Sales
0,0,4759.096031
1,2,6942.568678
2,6,8817.050891
3,7,5539.358418
4,8,6562.337612
5,9,5568.420918
6,10,8030.977041
7,11,7589.598214
8,12,5034.747182
9,13,5508.567394


In [24]:
df_test[df_test['Open'] == 0][['Store', 'Sales']].head()

,Store,Sales
543,702,0.0
676,878,0.0
840,1096,0.0
1399,702,0.0
1532,878,0.0


In [25]:
sample_csv['Sales'] = df_test['Sales']

In [26]:
sample_csv.to_csv(f'submision_baseline.csv', index=False)

In [27]:
sample_csv.head()

,Id,Sales
0,1,4759.096031
1,2,6942.568678
2,3,8817.050891
3,4,5539.358418
4,5,6562.337612


# Sumbit a la competición

In [28]:
sample_csv = pd.read_csv('dataset/rossmann/sample_submission.csv')
sample_csv['Sales'] = y_pred_test
sample_csv.head()

sample_csv.to_csv(f'submision_{log_output}-{min_child_samples}-{n_estimators}-{learning_rate}.csv', index=False)

In [29]:
!kaggle competitions submit -c rossmann-store-sales -f "submision_{log_output}-{min_child_samples}-{n_estimators}-{learning_rate}.csv" -m "submision_{log_output}-{min_child_samples}-{n_estimators}-{learning_rate}"

Successfully submitted to Rossmann Store Sales



  0%|          | 0.00/996k [00:00<?, ?B/s]
  1%|          | 8.00k/996k [00:00<00:13, 73.4kB/s]
 10%|#         | 104k/996k [00:00<00:04, 223kB/s]  
 13%|#2        | 128k/996k [00:00<00:06, 131kB/s]
 20%|##        | 200k/996k [00:01<00:03, 222kB/s]
 28%|##8       | 280k/996k [00:01<00:02, 329kB/s]
 33%|###2      | 328k/996k [00:01<00:04, 156kB/s]
 37%|###6      | 368k/996k [00:02<00:06, 105kB/s]
 40%|####      | 400k/996k [00:03<00:06, 91.0kB/s]
 43%|####2     | 424k/996k [00:03<00:05, 100kB/s] 
 45%|####4     | 448k/996k [00:03<00:04, 113kB/s]
 48%|####8     | 480k/996k [00:03<00:03, 136kB/s]
 53%|#####3    | 528k/996k [00:04<00:03, 125kB/s]
 57%|#####7    | 568k/996k [00:04<00:02, 159kB/s]
 60%|######    | 600k/996k [00:04<00:04, 95.8kB/s]
 63%|######3   | 632k/996k [00:04<00:03, 115kB/s] 
 66%|######5   | 656k/996k [00:05<00:03, 96.7kB/s]
 68%|######8   | 680k/996k [00:05<00:03, 84.2kB/s]
 71%|#######   | 704k/996k [00:05<00:02, 99.9kB/s]
 72%|#######2  | 720k/996k [00:06<00:03, 77.1